<a href="https://colab.research.google.com/github/rhMyCode/Kaggle/blob/main/House-Prices-Advanced-Regression-Techniques/Model_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("my--- colab")

my--- colab


In [ ]:
print('test')

test


In [ ]:

import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Conv2D, Flatten, MaxPool2D
from keras.models import Model